# Starter API Access Tunnel
---

## Imports

In [143]:
# Dependencies
import datetime as dt
import requests
import pandas as pd
from pprint import pprint

import api_keys

In [144]:
fema_decl = pd.read_csv('DisasterDeclarationsSummaries.csv', low_memory = False)
fema_decl.head(25)

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id
0,FM-5530-NV,5530,NV,FM,2024-08-12T00:00:00.000Z,2024,Fire,GOLD RANCH FIRE,0,0,...,99031,Washoe (County),24123,NaN,2024081201,9,R,2024-08-27T18:22:14.800Z,5d07e7c51bb300bfbec94a699a1e1ab1d61a97cd,f15a7a79-f1c3-41bb-8a5c-c05fbae34423
1,FM-5529-OR,5529,OR,FM,2024-08-09T00:00:00.000Z,2024,Fire,LEE FALLS FIRE,0,0,...,99067,Washington (County),24122,NaN,2024081001,10,R,2024-08-27T18:22:14.800Z,ae87cf3c6ed795015b714af7166c7c295b2b67c7,09e3f81a-5e16-4b72-b317-1c64e0cfa59c
2,FM-5528-OR,5528,OR,FM,2024-08-06T00:00:00.000Z,2024,Fire,ELK LANE FIRE,0,0,...,99031,Jefferson (County),24116,NaN,2024080701,10,R,2024-08-27T18:22:14.800Z,432cf0995c47e3895cea696ede5621b810460501,59983f89-30bf-4888-b21b-62e8d57d9aac
3,FM-5527-OR,5527,OR,FM,2024-08-02T00:00:00.000Z,2024,Fire,MILE MARKER 132 FIRE,0,0,...,99017,Deschutes (County),24111,NaN,2024080301,10,R,2024-08-27T18:22:14.800Z,2f21d90cb6bc64b0d4121aa3f18d852bbb4b11fa,8d13ecf0-bc2f-496b-8c9f-b2e73da832a0
4,FM-5522-CA,5522,CA,FM,2024-07-27T00:00:00.000Z,2024,Fire,BOREL FIRE,0,0,...,99029,Kern (County),24102,NaN,2024072701,9,R,2024-08-27T18:22:14.800Z,51ec819011c936f387edf10b9d5839b35419ca95,9b73e19b-d326-4992-8da1-7e658d97607c
5,FM-5521-WA,5521,WA,FM,2024-07-26T00:00:00.000Z,2024,Fire,RETREAT FIRE,0,0,...,99077,Yakima (County),24098,NaN,2024072601,10,R,2024-08-27T18:22:14.800Z,e205f5a4b56cadd811661c07fc8a8143b6c8c604,bc02a243-2458-4af3-84e3-b9928bd1d41d
6,FM-5520-ID,5520,ID,FM,2024-07-26T00:00:00.000Z,2024,Fire,GWEN FIRE,0,0,...,57340,Nez Perce Indian Reservation,24099,NaN,2024072602,10,R,2024-08-27T18:22:14.800Z,ba084caa4ee18c9c0c4383346e7669d4af4ab9b7,85cccdec-5e22-436f-a3e7-acd7340172cf
7,FM-5520-ID,5520,ID,FM,2024-07-26T00:00:00.000Z,2024,Fire,GWEN FIRE,0,0,...,99057,Latah (County),24099,NaN,2024072602,10,R,2024-08-27T18:22:14.800Z,0efc8eae4e51c80bcc84a928cea6a4d9f99420b9,d0e4282c-5d39-4af2-814c-2192d748f16c
8,FM-5520-ID,5520,ID,FM,2024-07-26T00:00:00.000Z,2024,Fire,GWEN FIRE,0,0,...,99069,Nez Perce (County),24099,NaN,2024072602,10,R,2024-08-27T18:22:14.800Z,b3b37b3287af4f410534ef8c33615c70f30e9e4c,b058f371-8e3e-49dc-bb08-aac48229a85f
9,FM-5519-CA,5519,CA,FM,2024-07-25T00:00:00.000Z,2024,Fire,PARK FIRE,0,0,...,99007,Butte (County),24097,NaN,2024072502,9,R,2024-08-27T18:22:14.800Z,b97481ee189819bedfc82da45902c285e97adac1,00045754-7992-448a-a8ae-181fb748a576


In [145]:
# Removing unwanted/unusable columns
fema_decl_clean = fema_decl[
    [
        'disasterNumber', 'declarationRequestNumber', 'declarationTitle', 'incidentType', 
        'declarationType', 'designatedArea', 'state', 'region', 'declarationDate', 
        'incidentBeginDate', 'tribalRequest', 'ihProgramDeclared', 'iaProgramDeclared', 
        'paProgramDeclared', 'hmProgramDeclared'
    ]
].copy()
fema_decl_clean['declarationDate'] = fema_decl_clean[
    'declarationDate'
    ].str.slice_replace(10)
fema_decl_clean['incidentBeginDate'] = fema_decl_clean[
    'incidentBeginDate'
    ].str.slice_replace(10)
fema_decl_clean

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67351,9,53010,FLOOD,Flood,DR,Statewide,TX,6,1953-06-19,1953-06-19,0,0,1,1,1
67352,8,53008,FLOOD,Flood,DR,Statewide,IA,7,1953-06-11,1953-06-11,0,0,1,1,1
67353,7,53009,TORNADO,Tornado,DR,Statewide,MA,1,1953-06-11,1953-06-11,0,0,1,1,1
67354,2,53003,TORNADO & HEAVY RAINFALL,Tornado,DR,Statewide,TX,6,1953-05-15,1953-05-15,0,0,1,1,1


In [146]:
# Unique FEMA Disaster Types
disaster_types = fema_decl_clean['incidentType'].unique()
pprint(disaster_types)

array(['Fire', 'Severe Storm', 'Straight-Line Winds', 'Flood',
       'Winter Storm', 'Hurricane', 'Tornado', 'Mud/Landslide',
       'Tropical Storm', 'Snowstorm', 'Earthquake', 'Coastal Storm',
       'Other', 'Severe Ice Storm', 'Biological', 'Dam/Levee Break',
       'Typhoon', 'Volcanic Eruption', 'Freezing', 'Toxic Substances',
       'Chemical', 'Terrorist', 'Drought', 'Human Cause',
       'Fishing Losses', 'Tsunami'], dtype=object)


In [147]:
disaster_natural = pd.DataFrame(
    {
        'incidentType': [
            'Fire', 'Severe Storm', 'Straight-Line Winds', 
            'Flood', 'Winter Storm', 'Hurricane', 'Tornado',
            'Mud/Landslide', 'Tropical Storm', 'Snowstorm', 
            'Earthquake', 'Coastal Storm', 'Severe Ice Storm', 
            'Typhoon', 'Volcanic Eruption', 'Freezing', 
            'Drought', 'Tsunami'
        ]
    }
)

In [149]:
# Filter for just natural disaster types
fema_disasters = pd.merge(fema_decl_clean, disaster_natural, how = 'right', on = 'incidentType')
# fema_disasters = fema_disasters.rename(
#     columns = {
#         'disasterNumber': 'disaster_number', 'designatedArea': 'area',
#         'declarationRequestNumber': 'request_number', 'declarationTitle': 'title',
#         'incidentType': 'disaster_type''

#     }
# )
display(fema_disasters)

,disasterNumber,declarationRequestNumber,declarationTitle,incidentType,declarationType,designatedArea,state,region,declarationDate,incidentBeginDate,tribalRequest,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared
0,5530,24123,GOLD RANCH FIRE,Fire,FM,Washoe (County),NV,9,2024-08-12,2024-08-11,0,0,0,1,1
1,5529,24122,LEE FALLS FIRE,Fire,FM,Washington (County),OR,10,2024-08-09,2024-08-08,0,0,0,1,1
2,5528,24116,ELK LANE FIRE,Fire,FM,Jefferson (County),OR,10,2024-08-06,2024-08-04,0,0,0,1,1
3,5527,24111,MILE MARKER 132 FIRE,Fire,FM,Deschutes (County),OR,10,2024-08-02,2024-08-02,0,0,0,1,1
4,5522,24102,BOREL FIRE,Fire,FM,Kern (County),CA,9,2024-07-27,2024-07-25,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59096,1967,11042,TSUNAMI WAVES,Tsunami,DR,Honolulu (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59097,1967,11042,TSUNAMI WAVES,Tsunami,DR,Maui (County),HI,9,2011-04-08,2011-03-11,0,0,0,1,1
59098,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Coos (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1
59099,1964,11029,TSUNAMI WAVE SURGE,Tsunami,DR,Curry (County),OR,10,2011-03-25,2011-03-11,0,0,0,1,1


In [134]:
# Counts of declaration types 
# DR = Major Disasters, EM = Emergency Declatation, FM = Fire Management)
# For more info see README (in progress)
fema_disasters['declarationType'].value_counts()

declarationType
DR    41116
EM    15991
FM     1994
Name: count, dtype: int64

In [142]:
# Counts of natural disaster types
fema_disasters['incidentType'].value_counts()

incidentType
Severe Storm           18399
Hurricane              13721
Flood                  11093
Fire                    3759
Snowstorm               3707
Severe Ice Storm        2942
Tornado                 1623
Drought                 1292
Tropical Storm          1047
Coastal Storm            637
Freezing                 301
Earthquake               228
Typhoon                  130
Winter Storm             117
Volcanic Eruption         51
Mud/Landslide             43
Tsunami                    9
Straight-Line Winds        2
Name: count, dtype: int64

In [135]:
# Validating no NaNs
fema_disasters.count()

disasterNumber              59101
declarationRequestNumber    59101
declarationTitle            59101
incidentType                59101
declarationType             59101
designatedArea              59101
state                       59101
region                      59101
declarationDate             59101
incidentBeginDate           59101
tribalRequest               59101
ihProgramDeclared           59101
iaProgramDeclared           59101
paProgramDeclared           59101
hmProgramDeclared           59101
dtype: int64

In [136]:
# Filter: Natural disasters per State
fema_disasters['state'].value_counts()

state
TX    4779
FL    2635
KY    2554
MO    2534
LA    2398
GA    2333
VA    2308
OK    2305
NC    2110
PR    1947
MS    1777
IA    1719
KS    1607
AL    1603
CA    1557
TN    1532
AR    1472
MN    1406
NE    1336
ND    1280
IN    1279
NY    1278
SD    1181
WV    1148
PA    1118
IL    1109
OH    1102
SC    1039
WA     902
WI     725
MI     595
NJ     556
OR     536
CO     531
MT     507
ME     426
MD     399
NM     380
VT     377
MA     365
NH     300
ID     276
AZ     271
CT     245
AK     211
NV     198
UT     183
RI     106
HI      97
WY      86
VI      79
MP      67
AS      66
MH      53
DE      47
FM      31
GU      20
DC      19
PW       1
Name: count, dtype: int64

In [141]:
# Filter: Natural disasters per hit area
fema_disasters['designatedArea'].value_counts()

designatedArea
Washington (County)                          510
Jefferson (County)                           462
Statewide                                    450
Jackson (County)                             450
Franklin (County)                            441
                                            ... 
Southern Ute Indian Reservation                1
Ute Mountain Indian Reservation (Also NM)      1
Nondalton (ANV/ANVSA)                          1
Salt River Pima-Maricopa Indian Community      1
Namorik (County-equivalent)                    1
Name: count, Length: 2201, dtype: int64

## Geoapify Geocoding API

In [ ]:
geoapify_base = 'https://api.geoapify.com/v1/geocode/search?'
geoapify_params = {
    'apiKey': api_keys.geoapify_key
    }
